In [39]:
import pandas as pd
import glob
import re
# Create an empty list to store the A counts
a_counts = []
total_a_count = 0
'''
availibility_data = {
        'Monday':[0,0,0,0,0,0,0,0,0],
        'Tuesday':[0,0,0,0,0,0,0,0,0],
        'Wednesday':[0,0,0,0,0,0,0,0,0],
        'Thursday':[0,0,0,0,0,0,0,0,0],
        'Friday':[0,0,0,0,0,0,0,0,0]}
#availibility_df = pd.DataFrame(availibility_data)
'''

def insert_name_by_preference(names_list, name, preference):
    # Define the preference order
    preference_order = {
        'B1': 0,
        'A1': 1,
        'A2': 2,
        'B2': 3,
        'B': 4,
        'A': 5,
        'D1': 6,
        'D2': 7,
        'N': 8
    }

    # Get the index to insert the name based on its preference
    insert_index = len(names_list)
    for index, existing_name in enumerate(names_list):
        if preference_order[existing_name[1]] > preference_order[preference]:
            insert_index = index
            break

    # Insert the name at the calculated index
    names_list.insert(insert_index, (name, preference))
    return names_list


data = [[[] for _ in range(5)] for _ in range(9)]
columns1 =   ["10-11pm","11-12pm","12-1pm","1-2pm","2-3pm","3-4pm","4-5pm","5-6pm","6-7pm"]
availibility_df = pd.DataFrame(data, columns=['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY'], index=columns1)



names_list = []
people_dict = {}
a_count=0

for col in range(0, 5):
    for row in range(0, 9):                                         #CHANGE 9 to 10                                        
    # Loop through each CSV file
        for filename in glob.glob("*.csv"):
            # Read in the CSV file using pandas
            df = pd.read_csv(filename)
            people_dict[df.columns[0]]= 3 #Sets 3 hours to each person
            # iterate through each file of a particular cell.
            cell_val = df.iloc[row, col+1]
            # If "A" Or "B" occurs in the cell, then it adds the name to a dataframe
            if "B" in str(cell_val) or "A" in str(cell_val):
                person_name = df.columns[0]
                #print(f"person name: {df.columns[0]}")
                current_names = str(availibility_df.iloc[row,col]).replace("0,", "")
                a_count+=1
                cell_val = ''.join([char for char in cell_val if char != 'Y']) #removes Y from any string.
                cell_val = re.sub(r'(?<=\d)\d$', '', cell_val)                 #removes last digit if two digits are present.
                availibility_df.iloc[row,col] = insert_name_by_preference(availibility_df.iloc[row,col], df.columns[0], cell_val)
                
        a_count=0
        total_a_count = 0



#finds the total numbers of hours a person is available each week and stores them in a dictionary.
def find_total_available_hours(df):
    name_counts = {}
    for _, row in df.iterrows():
        for cell in row:
            for name, _ in cell:
                # Increment the count for the name in the dictionary
                name_counts[name] = name_counts.get(name, 0) + 1

    return name_counts




# Call the function with the dataframe and people_dict
print(find_total_available_hours(availibility_df))
#print(people_dict)
#print(availibility_df.iloc[4,4])
counts_df = availibility_df.astype(str).applymap(lambda x: len(x.split(',')))
availibility_df

#availibility_df.to_csv('availibility.csv',index=columns)

{'Sneha': 8, 'Sriya': 21, 'Gunjan': 18, 'Nandini': 6, 'Alicia W': 15, 'Nikhil': 17, 'Tiffany': 11, 'Luisa': 14, 'Sahil': 9, 'Parshva': 8, 'Zeel ': 7, 'Ravinder': 4}


,MONDAY,TUESDAY,WEDNESDAY,THURSDAY,FRIDAY
10-11pm,"[(Sneha, B1), (Sriya, B1)]","[(Sriya, A)]","[(Gunjan, B)]",[],"[(Sriya, B), (Nandini, A)]"
11-12pm,"[(Sneha, B2), (Sriya, B2)]","[(Alicia W, B1), (Sriya, A)]","[(Nikhil, A), (Tiffany, A), (Gunjan, A)]","[(Luisa, B1)]","[(Nikhil, B1), (Luisa, B1), (Sriya, B), (Nandi..."
12-1pm,[],"[(Alicia W, B1), (Sahil, B1), (Nikhil, B), (Na...","[(Luisa, B1), (Sahil, B1), (Nandini, A), (Nikh...",[],"[(Luisa, B1), (Nikhil, A)]"
1-2pm,"[(Sahil, B1)]","[(Alicia W, B1), (Sahil, B1), (Sriya, A1), (Pa...","[(Alicia W, B1), (Sahil, B1), (Parshva, B2), (...","[(Sahil, B1), (Nandini, A)]","[(Alicia W, B1), (Luisa, B1), (Nikhil, A)]"
2-3pm,"[(Alicia W, B1), (Sneha, B1), (Nikhil, B), (Sr...","[(Alicia W, B1), (Sneha, B1), (Sriya, A2), (Ti...","[(Alicia W, B1), (Sriya, A2), (Tiffany, B2), (...","[(Alicia W, B1), (Sriya, B), (Nikhil, A), (Gun...","[(Alicia W, B1), (Luisa, B1), (Sriya, B1), (Ni..."
3-4pm,"[(Alicia W, B1), (Ravinder, A2), (Sneha, B2), ...","[(Alicia W, B1), (Sneha, B2), (Sriya, A), (Gun...",[],"[(Ravinder, A2), (Sriya, B), (Nikhil, A), (Gun...","[(Alicia W, B1), (Sriya, B2), (Nikhil, A), (Gu..."
4-5pm,"[(Parshva, B1), (Ravinder, A2), (Luisa, B), (G...","[(Nikhil, B1), (Parshva, B1), (Gunjan, A)]","[(Parshva, B1), (Luisa, A)]","[(Sneha, B1), (Parshva, B1), (Ravinder, A2), (...","[(Alicia W, B1), (Sriya, B)]"
5-6pm,"[(Sneha, B1), (Parshva, B2), (Luisa, B), (Gunj...","[(Nikhil, B1), (Gunjan, A)]","[(Luisa, A), (Gunjan, A)]","[(Parshva, B2), (Luisa, A)]","[(Zeel , B1), (Sriya, B)]"
6-7pm,"[(Sahil, B2), (Luisa, B), (Sriya, B), (Zeel , B)]","[(Alicia W, B1), (Zeel , B1), (Nikhil, B), (Ti...","[(Zeel , B1), (Sahil, B2), (Sriya, B), (Nikhil...","[(Tiffany, B1), (Sahil, B2), (Zeel , B), (Luis...","[(Zeel , B2), (Sriya, B), (Tiffany, A)]"


In [40]:
#print(people_dict)
#print(find_total_available_hours(availibility_df, people_dict))


labels = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
columns = ["10-11AM", "11-12PM", "12-1PM", "1-2PM", "2-3PM", "3-4PM", "4-5PM", "5-6PM", "6-7PM"]
finalzied_df = pd.DataFrame(columns=labels, index=columns)
#availibility_df.to_csv('Availibilities.csv',index=columns)


def is_cell_empty(row, column):
    cell_value = finalzied_df.iloc[row, column]
    return pd.isna(cell_value) or cell_value == '' or cell_value == []

#This function rearranges the list, priortizes people's prefrences and then the dictionary's values.
def custom_sort(item):
    name, preference = item
    # Create a dictionary to store the preference order
    preference_order = {'A': 6,'A1': 5,'A2': 4, 'B': 3, 'B1': 2, 'B2': 1}
    return (preference_order[preference], find_total_available_hours(availibility_df)[name])
    
# Sort the list using the custom sorting function
#sorted_list = sorted(data_list, key=custom_sort)    

#TESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTEST
#ESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTEST
#ESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTEST
# Flatten the dataframe and store the row and column index along with the length of the list of tuples
flattened_data = [(len(availibility_df.iloc[i, j]), (i, j)) for i in range(availibility_df.shape[0]) for j in range(availibility_df.shape[1])]

# Sort the flattened data based on the length of the list of tuples
sorted_data = sorted(flattened_data, key=lambda x: x[0])

# Iterate through the sorted data and access the original cell using the stored row and column indices
for length, (row, col) in sorted_data:
    cell_data = availibility_df.iloc[row, col]
    print(f"Cell with {length} tuples: {cell_data}")
#TESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTEST
#TESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTESTEST
for col in range(0, 5):
    for row in range(0, 9):
        while True:
            if is_cell_empty(row, col):
                if availibility_df.iloc[row][col]:  # Check if the list is not empty
                    print(f"choosing from: {availibility_df.iloc[row][col]}")
                    print(f"sorted:        {sorted(availibility_df.iloc[row][col], key=custom_sort)}")
                    temp_person = availibility_df.iloc[row][col][0][0] #can be replaced with some better logic, and in a function
                    if people_dict[temp_person] > 0:
                        finalzied_df.iloc[row][col] = temp_person
                        people_dict[temp_person] = people_dict[temp_person] - 1
                        print(f"Assigning {temp_person}")
                        # print(people_dict)
                        break  # Exit the while loop and proceed to the next iteration of the row
                    elif people_dict[temp_person] == 0:
                        print(f"Not hours left for {temp_person}")
                        # Create a new list without the first tuple
                        updated_list = availibility_df.iloc[row][col][1:]
                        # Update the cell value with the new list using the .iat accessor
                        availibility_df.iat[row, col] = updated_list
                else:
                    break  # Exit the while loop if the list in the cell is empty
            else:
                break  # Exit the while loop if the cell is not empty


#def assign_person():
    #resume coding from here, define this function
#availibility_df   
finalzied_df
finalzied_df.to_csv('Finalized Schedule.csv',index=columns)
#Assigning hours to each cell
# def assign_hours(availibility_df):
#     for col in range(0, 5):
#         for row in range(0, 9):
#             #finalzied_df.iloc[row,col] = assign_person()
        


#assign_hours(availibility_df)


Cell with 0 tuples: []
Cell with 0 tuples: []
Cell with 0 tuples: []
Cell with 0 tuples: []
Cell with 1 tuples: [('Sriya', 'A')]
Cell with 1 tuples: [('Gunjan', 'B')]
Cell with 1 tuples: [('Luisa', 'B1')]
Cell with 1 tuples: [('Sahil', 'B1')]
Cell with 2 tuples: [('Sneha', 'B1'), ('Sriya', 'B1')]
Cell with 2 tuples: [('Sriya', 'B'), ('Nandini', 'A')]
Cell with 2 tuples: [('Sneha', 'B2'), ('Sriya', 'B2')]
Cell with 2 tuples: [('Alicia W', 'B1'), ('Sriya', 'A')]
Cell with 2 tuples: [('Luisa', 'B1'), ('Nikhil', 'A')]
Cell with 2 tuples: [('Sahil', 'B1'), ('Nandini', 'A')]
Cell with 2 tuples: [('Parshva', 'B1'), ('Luisa', 'A')]
Cell with 2 tuples: [('Alicia W', 'B1'), ('Sriya', 'B')]
Cell with 2 tuples: [('Nikhil', 'B1'), ('Gunjan', 'A')]
Cell with 2 tuples: [('Luisa', 'A'), ('Gunjan', 'A')]
Cell with 2 tuples: [('Parshva', 'B2'), ('Luisa', 'A')]
Cell with 2 tuples: [('Zeel ', 'B1'), ('Sriya', 'B')]
Cell with 3 tuples: [('Nikhil', 'A'), ('Tiffany', 'A'), ('Gunjan', 'A')]
Cell with 3 tuples

In [2]:
import pandas as pd
import numpy as np

data = {
    'A': [1, np.nan, '', []],
    'B': [np.nan, 2, 'text', [1, 2]]
}

df = pd.DataFrame(data)



# Check if the cell at row 0 and column 'A' is empty
is_empty = is_cell_empty(0, 'A')
print(f"Is the cell at row 0 and column 'A' empty? {is_empty}")

# Check if the cell at row 1 and column 'A' is empty
is_empty = is_cell_empty(1, 'A')
print(f"Is the cell at row 1 and column 'A' empty? {is_empty}")

# Check if the cell at row 2 and column 'A' is empty
is_empty = is_cell_empty(2, 'A')
print(f"Is the cell at row 2 and column 'A' empty? {is_empty}")

# Check if the cell at row 3 and column 'A' is empty
is_empty = is_cell_empty(3, 'A')
print(f"Is the cell at row 3 and column 'A' empty? {is_empty}")


NameError: name 'is_cell_empty' is not defined

In [8]:
#Assign
def find_hours(input_str, my_dict):
    if input_str in my_dict:
        return my_dict[input_str]
    else:
        return None

def find_smallest_hours(keys_to_compare,data):
    # Set the initial minimum value and key
    min_val = None
    min_key = None

    # Loop over the keys to compare
    for key in keys_to_compare:
        # Get the value for the current key
        val = data[key]
        # If this is the first key, set the initial minimum value and key
        if min_val is None:
            min_val = val
            min_key = key
        # Otherwise, compare the value with the current minimum value
        else:
            if val < min_val:
                min_val = val
                min_key = key
    return min_key,min_val

#def remove_person_from_df(string,person_name):
#    return ",".join([word for word in string.split(",") if word != person_name])

def delete_string_from_dataframe(df, target_string):
    for column in df.columns:
        df[column] = df[column].apply(lambda x: ','.join([word.strip() for word in x.split(',') if word.strip() != target_string]))
    return df

#print(finalzied_df)

smallest_cell_row=0
smallest_cell_col=0
smallest_cell_value=[]
for iterations in range(10):
    for smallest_value in range(10): #Can change it to largest possible value of the matrix
        for col in range(1, 6):
            for row in range(0, 9):
                cell_values = availibility_df.iloc[row, col].split(",")
                #print(f"Trying to fill: {row} x {col}: with {cell_values}")
                #print(f"smallest_value: {smallest_value}")

                if smallest_value == len(cell_values) and cell_values !=['']:
                    print(f"Available Guards for {row} row, {col} col: {cell_values} ")
                    if find_smallest_hours(cell_values,people_dict)[1]>0:
                        cell_temp = finalzied_df.iloc[row,col-1]
                        if pd.isna(cell_temp):
                            print(f"Assigning: {find_smallest_hours(cell_values,people_dict)[0]}")
                            assign_this_Person = find_smallest_hours(cell_values,people_dict)[0]
                            finalzied_df.iloc[row,col-1]= str(assign_this_Person)
                            people_dict[str(assign_this_Person)] -= 1
                            if find_smallest_hours(cell_values,people_dict)[1] == 0:
                                #print(f"Zero hours left for: {assign_this_Person}")
                                del people_dict[assign_this_Person]
                                availibility_df = delete_string_from_dataframe(availibility_df,assign_this_Person)
                            print(f"New Dict: {people_dict}")

finalzied_df.insert(0, 'Busch Hours', ["10-11AM", "11-12PM", "12-1PM", "1-2PM", "2-3PM", "3-4PM", "4-5PM", "5-6PM", "6-7PM"])
finalzied_df.to_csv('Finalized Schedule.csv',index=False)

NameError: name 'cell_values' is not defined

In [14]:
availibility_df  

""


In [ ]:
import pandas as pd
import random

# Define the strings to choose from
strings = ["YA", "YB", "YD", "B", "D", "N"]

# Define the number of rows and columns in the CSV file
num_rows = 9
num_cols = 5

# Create a list of randomly selected string values with 1's and 2's for each row and column
data = []
for i in range(num_rows):
    row = []
    for j in range(num_cols):
        s = random.choice(strings)
        if s != "N":
            num1 = 0
            num2 = 0
            if random.random() < 0.5:
                num1 = 1
            else:
                num2 = 1
            s = s + num1 * "1" + num2 * "2"
        row.append(s)
    data.append(row)

# Add 1 to three random strings and 2 to three other random strings
for i in range(3):
    row = random.randint(0, num_rows-1)
    col = random.randint(0, num_cols-1)
    if data[row][col] != "N":
        data[row][col] += "1"
for i in range(3):
    row = random.randint(0, num_rows-1)
    col = random.randint(0, num_cols-1)
    if data[row][col] != "N" and "1" not in data[row][col] and "2" not in data[row][col]:
        data[row][col] += "2"

fileName = 'person_10.csv'
# Create a Pandas DataFrame from the data and write it to a CSV file
df = pd.DataFrame(data)
df.to_csv(fileName, index=False)

# Print the contents of the CSV file to verify that it was saved correctly
with open(fileName, 'r') as f:
    print(f.read())


'''
#Extracode from assigning logic
 #print(f"Available Guards for {row} row, {col} col: {cell_values} ")
                #print(f"Their hours: {find_smallest_hours(cell_values,people_dict)[1]}")
                print(f"New Dict: {people_dict}")
                print(f"Cell Values: {cell_values}")
                if find_smallest_hours(cell_values,people_dict)[1]>0:
                    #print(f"Assigning: {find_smallest_hours(cell_values,people_dict)[0]}")
                    assign_this_Person = find_smallest_hours(cell_values,people_dict)[0]
                    finalzied_df.iloc[row,col]= str(assign_this_Person)
                    people_dict[str(assign_this_Person)] -= 1
                    if find_smallest_hours(cell_values,people_dict)[1] == 0:
                        print(f"Zero hours left for: {assign_this_Person}")
                        del people_dict[assign_this_Person]
                        availibility_df = delete_string_from_dataframe(availibility_df,assign_this_Person)
                    print(f"New Dict: {people_dict}")
                else:
                    print("Error handeled")
'''                

: 

In [ ]:
fileName = 'Finalzied Schedule.csv'
# Create a Pandas DataFrame from the data and write it to a CSV file
df = pd.DataFrame(data)
df.to_csv(fileName, index=False)